In [3]:
!pip install geopandas

  Attempting uninstall: click
    Found existing installation: click 7.1.2
    Uninstalling click-7.1.2:
      Successfully uninstalled click-7.1.2


In [8]:
!pip install fastparquet

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
conda-repo-cli 1.0.4 requires pathlib, which is not installed.
scipy 1.6.2 requires numpy<1.23.0,>=1.16.5, but you have numpy 1.24.4 which is incompatible.

  Attempting uninstall: python-dateutil
    Found existing installation: python-dateutil 2.8.1
    Uninstalling python-dateutil-2.8.1:
      Successfully uninstalled python-dateutil-2.8.1
  Attempting uninstall: numpy
    Found existing installation: numpy 1.20.1
    Uninstalling numpy-1.20.1:
      Successfully uninstalled numpy-1.20.1
  Attempting uninstall: pandas
    Found existing installation: pandas 1.2.4
    Uninstalling pandas-1.2.4:
      Successfully uninstalled pandas-1.2.4


In [ ]:
conda install -c conda-forge fiona

In [4]:
import pandas as pd
import geopandas as gpd
from shapely.geometry import LineString
import numpy as np

In [9]:
# Read the data files
properties = pd.read_parquet('gnaf_prop.parquet', engine='fastparquet')
# properties = gpd.read_parquet('gnaf_prop.parquet')

In [12]:
properties.dtypes()

TypeError: 'Series' object is not callable

In [10]:
properties.head(10)

,gid,gnaf_pid,street_locality_pid,locality_pid,alias_principal,primary_secondary,building_name,lot_number,flat_number,level_number,...,confidence,legal_parcel_id,mb_2016_code,mb_2021_code,latitude,longitude,geocode_type,reliability,geom,schema
0,3115,GANSW705858629,NSW2878308,locc9618e9a6979,P,None,None,1,None,None,...,2,1/105203,10840311000,10840311000,-33.787478,151.182595,PROPERTY CENTROID,2,0101000020BB100000E599B8D1D7E562406816D312CCE4...,gnaf_202402
1,3120,GANSW705858630,NSW2878308,locc9618e9a6979,P,None,None,26,None,None,...,2,26/7045,10840312000,10840312000,-33.787164,151.183209,PROPERTY CENTROID,2,0101000020BB1000008215D0D9DCE562403E3335C7C1E4...,gnaf_202402
2,3123,GANSW705858631,NSW2878308,locc9618e9a6979,P,None,None,16,None,None,...,2,15/455712,10840311000,10840311000,-33.787375,151.182549,PROPERTY CENTROID,2,0101000020BB100000E8F3B470D7E562406A31BAB4C8E4...,gnaf_202402
3,3126,GANSW705858632,NSW2878308,locc9618e9a6979,P,None,None,27,None,None,...,2,27/7045,10840312000,10840312000,-33.787023,151.183146,PROPERTY CENTROID,2,0101000020BB1000009DEE2354DCE56240D5DAEA28BDE4...,gnaf_202402
4,3130,GANSW705858633,NSW2878308,locc9618e9a6979,P,None,None,1,None,None,...,2,1/963791,10840311000,10840311000,-33.787273,151.182502,PROPERTY CENTROID,2,0101000020BB1000005048190ED7E5624024456458C5E4...,gnaf_202402
5,3133,GANSW705858634,NSW2878308,locc9618e9a6979,P,None,None,28,None,None,...,2,28/7045,10840312000,10840312000,-33.786883,151.183083,PROPERTY CENTROID,2,0101000020BB1000006D1DDACFDBE56240D7980091B8E4...,gnaf_202402
6,3140,GANSW705858637,NSW2878308,locc9618e9a6979,P,None,None,1,None,None,...,2,1/955990,10840312000,10840312000,-33.786739,151.183020,PROPERTY CENTROID,2,0101000020BB1000002F88484DDBE562407404D3DDB3E4...,gnaf_202402
7,3143,GANSW705858639,NSW2830117,loc99a5db2a404b,P,None,None,4,None,None,...,2,4/19155,10421870000,10421870000,-33.777122,151.195697,PROPERTY CENTROID,2,0101000020BB1000004304DA2543E66240CEB327BB78E3...,gnaf_202402
8,3147,GANSW705858640,NSW2830117,loc99a5db2a404b,P,None,None,5,None,None,...,2,5/19155,10421870000,10421870000,-33.777076,151.195907,PROPERTY CENTROID,2,0101000020BB10000098380BDE44E662408218CF3D77E3...,gnaf_202402
9,3151,GANSW705858641,NSW2830117,loc99a5db2a404b,P,None,None,767,None,None,...,2,B/389424,10421950000,10421950000,-33.774313,151.203437,PROPERTY CENTROID,2,0101000020BB10000091C3588D82E662408C01A7B11CE3...,gnaf_202402


In [14]:
cadastre = gpd.read_file('./cadastre.gpkg')

AttributeError: module 'fiona' has no attribute 'path'

In [6]:
roads = gpd.read_file('roads.gpkg')

OSError: NotImplemented: Support for codec 'snappy' not built

In [ ]:
# Read the orientation notebook to understand the methodology
# (assuming it contains the logic for calculating orientation)

# Function to calculate the orientation/direction a property faces
def calculate_orientation(property_geometry, road_geometry):
    """
    Calculate which direction a property faces based on its boundary
    closest to the nearest road
    """
    # Get the boundary of the property that's closest to the road
    if property_geometry.geom_type == 'Polygon':
        # Get all edges of the polygon
        coords = list(property_geometry.exterior.coords)
        
        # Find the edge closest to the road
        min_distance = float('inf')
        closest_edge = None
        
        for i in range(len(coords) - 1):
            edge = LineString([coords[i], coords[i + 1]])
            distance = edge.distance(road_geometry)
            
            if distance < min_distance:
                min_distance = distance
                closest_edge = edge
        
        # Calculate the bearing/orientation of this edge
        if closest_edge:
            x1, y1 = closest_edge.coords[0]
            x2, y2 = closest_edge.coords[1]
            
            # Calculate angle in degrees
            angle = np.degrees(np.arctan2(y2 - y1, x2 - x1))
            
            # Normalize to 0-360 degrees
            angle = (angle + 360) % 360
            
            # Convert to cardinal direction
            return get_cardinal_direction(angle)
    
    return None

def get_cardinal_direction(angle):
    """
    Convert angle to cardinal direction
    """
    directions = ['North', 'North-East', 'East', 'South-East', 
                  'South', 'South-West', 'West', 'North-West']
    
    # Divide 360 degrees into 8 segments (45 degrees each)
    index = round(angle / 45) % 8
    
    return directions[index]

# Spatial join properties with cadastre to get property boundaries
properties_with_cadastre = gpd.sjoin(properties, cadastre, 
                                      how='left', predicate='within')

# Spatial join with roads to find nearest road for each property
properties_with_roads = gpd.sjoin_nearest(properties_with_cadastre, roads, 
                                           how='left', max_distance=100)

# Calculate orientation for each property
orientations = []

for idx, row in properties_with_roads.iterrows():
    prop_geom = row.geometry
    
    # Get the corresponding road geometry
    if pd.notna(row['index_right']):
        road_geom = roads.loc[row['index_right'], 'geometry']
        orientation = calculate_orientation(prop_geom, road_geom)
    else:
        orientation = 'Unknown'
    
    orientations.append({
        'address': row.get('address', 'N/A'),  # Adjust column name as needed
        'orientation': orientation
    })

# Create output dataframe
output_df = pd.DataFrame(orientations)

# Remove duplicates (one row per property)
output_df = output_df.drop_duplicates(subset=['address'])

# Save to file
output_df.to_csv('property_orientations.csv', index=False)
output_df.to_parquet('property_orientations.parquet', index=False)

print(f"Processed {len(output_df)} properties")
print(output_df.head())
print("\nOrientation distribution:")
print(output_df['orientation'].value_counts())